In [1]:
from bokeh.io import output_notebook, show, curdoc
from bokeh.plotting import figure
from bokeh.models import MultiChoice, CustomJS, RangeSlider, HoverTool, ColumnDataSource, Column, Slider
from bokeh.layouts import column
from bokeh.embed import file_html
from bokeh.plotting import figure, show
from bokeh.transform import cumsum
from bokeh.resources import CDN

from math import pi
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/annual-number-of-deaths-by-cause.csv', usecols=np.arange(2,34))
df.columns = [col.replace("- Sex: Both - Age: All Ages (Number)", '').replace("Deaths -", '') for col in df.columns]
df = df.drop(df[df.index > 29].index)
df.head()

,Year,Meningitis,Alzheimer's disease and other dementias,Parkinson's disease,Nutritional deficiencies,Malaria,Drowning,Interpersonal violence,Maternal disorders,HIV/AIDS,...,Diabetes mellitus,Chronic kidney disease,Poisonings,Protein-energy malnutrition,Road injuries,Chronic respiratory diseases,Cirrhosis and other chronic liver diseases,Digestive diseases,"Fire, heat, and hot substances",Acute hepatitis
0,1990,2159,1116,371,2087,93,1370,1538,2655,34,...,2108,3709,338,2054,4154,5945,2673,5005,323,2985
1,1991,2218,1136,374,2153,189,1391,2001,2885,41,...,2120,3724,351,2119,4472,6050,2728,5120,332,3092
2,1992,2475,1162,378,2441,239,1514,2299,3315,48,...,2153,3776,386,2404,5106,6223,2830,5335,360,3325
3,1993,2812,1187,384,2837,108,1687,2589,3671,56,...,2195,3862,425,2797,5681,6445,2943,5568,396,3601
4,1994,3027,1211,391,3081,211,1809,2849,3863,63,...,2231,3932,451,3038,6001,6664,3027,5739,420,3816


In [3]:
colours, source = [], []

In [4]:
# Assigns a colour to each disease
from bokeh.palettes import Turbo256
split = 0
for i in range(len(df.columns[1:])):
    colours.append(Turbo256[split])
    split += int(len(Turbo256) / (len(df.columns)) - 1)

In [5]:
slider = Slider(start=min(df["Year"]), end=max(df["Year"]), value=min(df["Year"]), step=1, title="Year")
p = figure(width=700, height=400, title="Death Causes By Proportion", tooltips = "@Disease @Percentage%")

In [6]:
# Create custom data to be displayed onto the chart i.e gen_data(1990) = CDS({"Disease: [" Meningitis ", " Alzheimer's disease and other dementias ",...], "Deaths":[2159, 1116...],...})
def gen_data(year):
    idx = year - 1990
    data = {
    "Disease": df.columns[1:].to_list(),
    "Deaths": df.loc[idx][1:].reset_index(drop=True).to_list(),
    "Percentage": (df.loc[idx][1:] / sum(df.loc[idx][1:]) * 100).reset_index(drop=True).to_list(),
    "Angle": (df.loc[idx][1:] / sum(df.loc[idx][1:]) * 2*pi).reset_index(drop=True).to_list(),
    "Colour": colours}
    return ColumnDataSource(data=data)

In [7]:
# source = [{"Disease": [" Meningitis ", " Alzheimer's disease and other dementias ",...], "Deaths": [2159, 1116...],}
# Data for each year is stored as separate elements (dictionaries) inside of the source list
for i in df["Year"]:
    tmp_dict = {}
    for j in gen_data(i).data.keys():
        tmp_dict[j] = gen_data(i).data[j]
    source.append(tmp_dict)

In [8]:
plot_source = ColumnDataSource(source[0])

In [9]:
p.wedge(x=0, y=0, radius=0.5, start_angle=cumsum('Angle', include_zero=True), end_angle=cumsum('Angle'), line_color="white", fill_color='Colour', source=plot_source) 

GlyphRenderer(id='p1602', ...)

In [21]:
p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None
p.outline_line_color = None
p.background_fill_color = "#262424"
p.title.text_font_size = '14pt'

p.toolbar.active_drag = None
p.toolbar_location = None

In [22]:
callback = CustomJS(args=dict(plot_source=plot_source, source=source), code="""
    const year_select = cb_obj.value
    const source_list = source
    plot_source.data = source_list[year_select - 1990]
""")

In [23]:
slider.js_on_change('value', callback)

layout = column(p, slider)
curdoc().theme = 'dark_minimal'

output_notebook()

show(layout)

Loading BokehJS ...

In [17]:
html = file_html(layout, CDN)
file_path = "deathcause.html"
with open(file_path, "w", encoding="utf-8") as file:
    file.write(html)